In [44]:
from collections import defaultdict
import numpy as np
import pandas as pd 
from sklearn.utils import shuffle

In [45]:
# load text

def load_train_data(bow, data_path):
    reviews = []
    f = open(data_path, "r")
    for line in f:
        review = line.split(" ")
        reviews.append(review)
        for word in review:
            if word not in bow:
                bow[word] = len(bow)
#             else:
#                 bow[word] = bow[word] + 1
    return reviews
                
                
def load_validation_data(data_path):
    reviews = []
    f = open(data_path, "r")
    for line in f:
        review = line.split(" ")
        reviews.append(review)
    return reviews
        
def unigram_BOW(reviews, bow, label):
    raw = []
    length = len(bow)
    for review in reviews:
        cur_bow = np.zeros(length)
        for word in review:
            if word in bow:
                cur_bow[bow[word]] += 1
            else:
                cur_bow[bow['UNKNOWN']] += 1
        raw.append([cur_bow, label])
        
    return raw
                
            
train_decep = "./train/deceptive.txt"
train_truth = "./train/truthful.txt"
validation_decep = "./validation/deceptive.txt"
validation_truth = "./validation/truthful.txt"

uni_bow = {}

deceptive_reviews_train  = load_train_data(uni_bow, train_decep)
truthful_reviews_train = load_train_data(uni_bow, train_truth)
uni_bow['UNKNOWN'] = len(uni_bow)

deceptive_reviews_val = load_validation_data(validation_decep)
truthful_reviews_val = load_validation_data(validation_truth)            
    
# generate unigram BOW
uni_train = unigram_BOW(deceptive_reviews_train, uni_bow, 0) + unigram_BOW(truthful_reviews_train, uni_bow, 1)
uni_val = unigram_BOW(deceptive_reviews_val, uni_bow, 0) + unigram_BOW(truthful_reviews_val, uni_bow, 1)

uni_train_X_y = shuffle(pd.DataFrame(uni_train, columns = ['X', 'y']))
uni_val_X_y = shuffle(pd.DataFrame(uni_val, columns = ['X', 'y']))


In [46]:
# train unigram NB 
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

uni_X_train, uni_y_train = list(uni_train_X_y.loc[:,'X']), list(uni_train_X_y.loc[:,'y'])
uni_X_val, uni_y_val = list(uni_val_X_y.loc[:,'X']), list(uni_val_X_y.loc[:,'y'])

clf = MultinomialNB()
clf.fit(uni_X_train, uni_y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
uni_y_pred = clf.predict(uni_X_val)

print("Accuracy:", metrics.accuracy_score(uni_y_val, uni_y_pred))

Accuracy: 0.91796875


In [52]:
# generate bigram BOW

def bigram_BOW_dict(reviews, bow):
    for review in reviews:
        for i in range(len(review)):
            if i == 0:
                key = '<s>' + ' ' + review[0]
            else:
                key = review[i] + ' ' + review[i-1]
            if key not in bow:
                bow[key] = len(bow)
                
def bigram_BOW(reviews, bow, label):
    raw = []
    length = len(bow)
    for review in reviews:
        cur_bow = np.zeros(length)
        for i in range(len(review)):
            if i == 0:
                key = '<s>' + ' ' + review[0]
            else:
                key = review[i] + ' ' + review[i-1]
            if key in bow:
                cur_bow[bow[key]] += 1
            else:
                cur_bow[bow['UNKNOWN']] += 1
        raw.append([cur_bow, label])
        
    return raw
        
bi_bow = {}
bigram_BOW_dict(deceptive_reviews_train, bi_bow)
bigram_BOW_dict(truthful_reviews_train, bi_bow)
bi_bow['UNKNOWN'] = len(bi_bow)

bi_train = bigram_BOW(deceptive_reviews_train, bi_bow, 0) + bigram_BOW(truthful_reviews_train, bi_bow, 1)
bi_val = bigram_BOW(deceptive_reviews_val, bi_bow, 0) + bigram_BOW(truthful_reviews_val, bi_bow, 1)

bi_train_X_y = shuffle(pd.DataFrame(bi_train, columns = ['X', 'y']))
bi_val_X_y = shuffle(pd.DataFrame(bi_val, columns = ['X', 'y']))

bi_X_train, bi_y_train = list(bi_train_X_y.loc[:,'X']), list(bi_train_X_y.loc[:,'y'])
bi_X_val, bi_y_val = list(bi_val_X_y.loc[:,'X']), list(bi_val_X_y.loc[:,'y'])

clf = MultinomialNB()
clf.fit(bi_X_train, bi_y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
bi_y_pred = clf.predict(bi_X_val)

print("Accuracy:", metrics.accuracy_score(bi_y_val, bi_y_pred))

Accuracy: 0.859375
